In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
import os

/Users/amber/Documents/Learnings_2024/Langraph_2025/langraph_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables from the .env file
load_dotenv()

True

In [3]:

# # Access the variables
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [4]:
# using langchain_google_genai
gemini_model = ChatGoogleGenerativeAI(model='gemini-1.5-flash',google_api_key=gemini_api_key)

In [5]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [8]:

class BlogState(TypedDict):

    title: str
    outline: str
    content: str
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = gemini_model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [9]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = gemini_model.invoke(prompt).content

    state['content'] = content

    return state

In [10]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [11]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

KeyboardInterrupt: 

In [ ]:
print(final_state['outline'])

In [ ]:
print(final_state['content'])